### Import the required libraries

In [1]:
import os
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

yf.pdr_override()

# Zipline imports
from zipline.utils.run_algo import load_extensions
from zipline.data import bundles
from zipline.utils.calendar_utils import get_calendar

from zipline.utils.calendar_utils import get_calendar
from zipline.api import(order_value,
                        symbol,
                        order_target_percent,
                        get_open_orders)
from zipline import run_algorithm
from zipline.utils.calendar_utils import get_calendar
from zipline.api import symbol # type: ignore

# Ignore Warnings  
import warnings
warnings.filterwarnings('ignore', category=Warning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import generate_signal as gs

2024-02-13 18:49:17.580124: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 18:49:18.524704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 18:49:18.525042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 18:49:18.626317: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 18:49:18.847395: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 18:49:18.850024: I tensorflow/core/platform/cpu_feature_guard.cc:1

Register bundle

In [5]:
bundle_name = 'sp50' # rename the bundle same as your ingested bundle if needed
                     # The default bundle name is 'sp50
bundles.register(
    bundle_name,
    bundles.csvdir.csvdir_equities(
        ['daily'],
        'Data/sp50'       # if you want to ingest your own data provide the path here
    ),
    calendar_name='NYSE',
)

bundles.ingest(bundle_name)

Data Ingestion

In [8]:
calendar_name = 'NYSE'
calendar = get_calendar(calendar_name)
bundle_data = bundles.load(bundle_name)
sids = bundle_data.asset_finder.sids
assets = bundle_data.asset_finder.retrieve_all(sids)
sp50tickers = [coin.symbol for coin in assets]

Define the time range for the data you want to analyze.<br>
The minimum time range should be ```50 days or 3 months```.The reason is the LSTM model neeeds past 50 days of data to predict the every next day(51st day).

In [13]:
START_DATE = pd.Timestamp('2018-01-01')  # change it to your desired time range
END_DATE = pd.Timestamp('2022-12-31')
BASE_CAPITAL = 100_000

### Bactesting

In [14]:
# %%capture   # uncomment the line if you want to skip the long output of the cell

def initialize(context):
    context.idx = 0
    context.tickers = sp50tickers  # List of tickers, slice it to your desired number of stocks to trade
    context.window = 50  # Number of days to look back for LSTM model, shoudn't change it
    context.buy_stocks = set()
    context.cash_pct = 0.25  # Percentage of cash to be invested in each stock , change it to your desired value

# Define the handle_data function
def handle_data(context, data):
    context.idx += 1
    if context.idx < context.window:
        return

    # Execute trades based on the generated signals
    for ticker in context.tickers:
        prices = data.history(symbol(ticker), "price", bar_count=context.window, frequency="1d")
        # Module is being called to generate signals
        signal = gs.group1_ensemble_model_signals(prices) # will return signals as 1, 0, -1
        open_orders = get_open_orders(ticker)

        # Change the order logic with your desired strategy, it is a demo order logic
        if ticker not in open_orders:
            if signal == 1 and ticker not in context.buy_stocks:
                if context.portfolio.cash > 0:      # Avoids negative cash to avoid short selling
                    order_value(symbol(ticker), context.portfolio.cash * context.cash_pct)  # Buy Signal
                    context.buy_stocks.add(ticker)

            elif signal == -1 and ticker in context.buy_stocks:
                order_target_percent(symbol(ticker), 0)  # Sell Signal
                context.buy_stocks.remove(ticker)

# Run the algorithm
results = run_algorithm(
    start=START_DATE,
    end=END_DATE,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=BASE_CAPITAL,
    benchmark_returns=None,
    data_frequency='daily',
    bundle=bundle_name
)

# remember to run the next cell to plot the performance

plot the results with 'quantstats' full report

In [ ]:
gs.plots(results)